교재 374쪽     LSTM을 쓰기 위한 IMDB 준비과정

In [1]:
#!pip install torchtext==0.6.0

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import FinanceDataReader as fdr

import torch
import torch.nn as nn
import torch.nn.functional as F
import time

import torchtext

In [3]:
TEXT=torchtext.data.Field(lower=True, fix_length=200, batch_first=False)
LABLE=torchtext.data.Field(sequential=False)

In [4]:
from torchtext.datasets import IMDB
train_data, test_data=IMDB.splits(TEXT, LABLE)


In [5]:
print(vars(train_data.examples[0]))

{'text': ['bromwell', 'high', 'is', 'a', 'cartoon', 'comedy.', 'it', 'ran', 'at', 'the', 'same', 'time', 'as', 'some', 'other', 'programs', 'about', 'school', 'life,', 'such', 'as', '"teachers".', 'my', '35', 'years', 'in', 'the', 'teaching', 'profession', 'lead', 'me', 'to', 'believe', 'that', 'bromwell', "high's", 'satire', 'is', 'much', 'closer', 'to', 'reality', 'than', 'is', '"teachers".', 'the', 'scramble', 'to', 'survive', 'financially,', 'the', 'insightful', 'students', 'who', 'can', 'see', 'right', 'through', 'their', 'pathetic', "teachers'", 'pomp,', 'the', 'pettiness', 'of', 'the', 'whole', 'situation,', 'all', 'remind', 'me', 'of', 'the', 'schools', 'i', 'knew', 'and', 'their', 'students.', 'when', 'i', 'saw', 'the', 'episode', 'in', 'which', 'a', 'student', 'repeatedly', 'tried', 'to', 'burn', 'down', 'the', 'school,', 'i', 'immediately', 'recalled', '.........', 'at', '..........', 'high.', 'a', 'classic', 'line:', 'inspector:', "i'm", 'here', 'to', 'sack', 'one', 'of', '

## 전처리


In [6]:
import string
for example in train_data.examples:
    text= [x.lower() for x in vars(example)["text"]]                             # 소문자로 변경
    text= [x.replace("<br" , "" )for x in text]
    text=["".join(c for c in s if c not in string.punctuation) for s in text]   #불용어 처리
    text=[s for s in text if s]                                                   #null이나 인식하지 못하는 값 삭제
    vars(example)["text"]=text

for example in test_data.examples:
    text= [x.lower() for x in vars(example)["text"]]                             # 소문자로 변경
    text= [x.replace("<br" , "" )for x in text]
    text=["".join(c for c in s if c not in string.punctuation) for s in text]   #불용어 처리
    text=[s for s in text if s]                                                   #null이나 인식하지 못하는 값 삭제
    vars(example)["text"]=text

In [7]:
import random
train_data, valid_data=train_data.split(random_state=random.seed(42), split_ratio=0.8)

In [8]:
TEXT.build_vocab(train_data, max_size=10000, min_freq=10, vectors =None)
LABLE.build_vocab(train_data, max_size=10000, min_freq=10, vectors =None)


In [9]:
embedding_dim=100
BATCH_SIZE=64
hidden_size=300

train_iterator, valid_iterator, test_iterator =torchtext.data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=BATCH_SIZE
)

In [10]:

        
class LSTMCell_Encoder(nn.Module):
    def __init__(self, input_dim, hidden_size):
        super(LSTMCell_Encoder, self).__init__()
        self.rnn = nn.LSTMCell(input_dim, hidden_size)

    def forward(self, inputs):
        bz = inputs.shape[1]
        ht = torch.zeros((bz, hidden_size))
        ct =torch.zeros((bz, hidden_size))
        for word in inputs:
            ht = self.lstm(word, (ht, ct))
        return ht
        
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.em = nn.Embedding(len(TEXT.vocab.stoi),embedding_dim)       #출력크기 embedding_dim=100
        self.rnn = LSTMCell_Encoder(embedding_dim, hidden_size)             #출력크기 hidden_size=300
        self.fc1 = nn.Linear(hidden_size, 256)
        self.fc2 = nn.Linear(256, 3)

    def forward(self, x):
        x = self.em(x)
        x = self.rnn(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [11]:
model = Net()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [21]:
def training(epoch, model, trainloader, validloader):
    correct = 0
    total = 0
    running_loss = 0

    model.train()
    for b in trainloader:
        x, y = b
        x, y = x.to, y.to
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        with torch.no_grad():
            y_pred = torch.argmax(y_pred, dim=1)
            correct += (y_pred == y).sum().item()
            total += y.size(0)
            running_loss += loss.item()
    epoch_loss = running_loss / len(trainloader.dataset)
    epoch_acc = correct / total

    valid_correct = 0
    valid_total = 0
    valid_running_loss = 0

    model.eval()
    with torch.no_grad():
        for b in validloader:
            x, y = b
            x, y = x.to, y.to
            y_pred = model(x)
            loss = loss_fn(y_pred, y)
            y_pred = torch.argmax(y_pred, dim=1)
            valid_correct += (y_pred == y).sum().item()
            valid_total += y.size(0)
            valid_running_loss += loss.item()

    epoch_valid_loss = valid_running_loss / len(validloader.dataset)
    epoch_valid_acc = valid_correct / valid_total

    print('epoch: ', epoch,
          'loss： ', round(epoch_loss, 3),
          'accuracy:', round(epoch_acc, 3),
          'valid_loss： ', round(epoch_valid_loss, 3),
          'valid_accuracy:', round(epoch_valid_acc, 3)
          )
    return epoch_loss, epoch_acc, epoch_valid_loss, epoch_valid_acc

In [22]:
epochs = 5
train_loss = []
train_acc = []
valid_loss = []
valid_acc = []

for epoch in range(epochs):
    epoch_loss, epoch_acc, epoch_valid_loss, epoch_valid_acc = training(epoch, model,    train_iterator,     valid_iterator)
    train_loss.append(epoch_loss)
    train_acc.append(epoch_acc)
    valid_loss.append(epoch_valid_loss)
    valid_acc.append(epoch_valid_acc)

end = time.time()
print(end-start)

AttributeError: 'tuple' object has no attribute 'to'